In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

In [ ]:
file_ch0 = "laser/output-wave_0.h5"
file_ch1 = "laser/output-TR_0_0.h5"
f_ch0 = h5py.File( file_ch0, 'r')
f_ch1 = h5py.File( file_ch1, 'r')
print ( f_ch0.keys() )
print ( f_ch1.keys() )

In [ ]:
dset_ch0 = f_ch0['Waveform']
dset_metadata_ch0 = f_ch0['Metadata']
print( dset_metadata_ch0 )
print ( dset_ch0 )

dset_ch1 = f_ch1['Waveform']
dset_metadata_ch1 = f_ch1['Metadata']
print( dset_metadata_ch1 )
print ( dset_ch1 )

In [ ]:
dset_metadata_ch0

In [ ]:
df_ch0 = pd.DataFrame( columns=('Event','Channel','Waveform') )
df_ch0['Event']   = dset_metadata_ch0[:,0].astype('int64')
df_ch0['Channel'] = dset_metadata_ch0[:,1].astype('int64')
for i in range( dset_ch0.shape[0] ):
    df_ch0[ 'Waveform' ].iloc[ i ] = dset_ch0[ i ]
df_ch0 = df_ch0.set_index( 'Event' )
df_ch0

In [ ]:
df_ch1 = pd.DataFrame( columns=('Event','Channel','Waveform') )
df_ch1['Event']   = dset_metadata_ch1[:,0].astype('int64')
df_ch1['Channel'] = dset_metadata_ch1[:,1].astype('int64')
for i in range( dset_ch1.shape[0] ):
    df_ch1[ 'Waveform' ].iloc[ i ] = dset_ch1[ i ]
df_ch1 = df_ch1.set_index( 'Event' )
df_ch1

In [ ]:
f_ch0.close()
f_ch1.close()

In [ ]:
df_all = df_ch0.join( df_ch1, how='inner', lsuffix='_ch0', rsuffix='_ch1' )
df_all

In [ ]:
events = df_all.index
events

In [ ]:
conv_X = 0.200 # ns
conv_Y = (2/4096)

i_evt = 10

event_ch0 = df_all.loc[ events[ i_evt ], 'Waveform_ch0' ]
print ( event_ch0 )
event_ch1 = df_all.loc[ events[ i_evt ], 'Waveform_ch1' ]
print ( event_ch1 )

baseline_ch0 = np.mean( event_ch0[:50] )
event_ch0_corr = event_ch0 - baseline_ch0
# event_ch0_corr = -event_ch0_corr
print ( event_ch0_corr )

baseline_ch1 = np.mean( event_ch1[:50] )
event_ch1_corr = event_ch1 - baseline_ch1
# event_ch1_corr = -event_ch1_corr
print ( event_ch1_corr )

fig = plt.figure( figsize=(8,6) )
X = conv_X * np.arange( 1024 ) 
plt.plot( X, ( event_ch0_corr * conv_Y ) )
plt.plot( X, ( event_ch1_corr * conv_Y ) )
plt.xlim(0,200)

In [ ]:
from scipy import interpolate

X_inter = np.linspace( X[0], X[1023], 50*len(X) )

f_ch0 = interpolate.interp1d(X, event_ch0_corr, 'cubic')
y_inter_ch0 = f_ch0( X_inter )

f_ch1 = interpolate.interp1d(X, event_ch1_corr, 'cubic')
y_inter_ch1 = f_ch1( X_inter )

fig = plt.figure( figsize=(8,6) )
plt.plot( X_inter, ( y_inter_ch0 * conv_Y ) )
plt.plot( X_inter, ( y_inter_ch1 * conv_Y ) )
plt.xlim(0,200)

In [ ]:
def corr_wf( event_ ):
    baseline_ = np.mean( event_[:50] )
    event_corr_ = event_ - baseline_
    # event_corr_ = -event_corr_
    return ( event_corr_ * conv_Y )

In [ ]:
idx_events = range( 0, 200, 10)
print ( list( idx_events ) )

# threshold_max_0 = 200 # ADC
# threshold_max_1 = 600 # ADC
threshold_max_0 = 0.08 # V
threshold_max_1 = 0.3 # V

fig, axes = plt.subplots( 1, 2, figsize=(16,6) )
for i_evt_ in idx_events:    
    event_ch0_ = df_all.loc[ events[ i_evt_ ], "Waveform_ch0" ]
    event_ch1_ = df_all.loc[ events[ i_evt_ ], "Waveform_ch1" ]
    event_ch0_corr_ = corr_wf( event_ch0_ )
    event_ch1_corr_ = corr_wf( event_ch1_ )

    max_ch0_ = np.max( -event_ch0_corr_ )
    max_ch1_ = np.max( -event_ch1_corr_ )

    if ( max_ch0_ < threshold_max_0 ) or ( max_ch1_ < threshold_max_1 ): continue

    axes[ 0 ].plot( X, event_ch0_corr_, label="event {:d}".format( i_evt_ ) )
    axes[ 1 ].plot( X, event_ch1_corr_, label="event {:d}".format( i_evt_ ) )
    
axes[ 0 ].legend( loc='best' )
axes[ 0 ].set_xlabel( "CH0", fontsize=16 )
axes[ 1 ].legend( loc='best' )
axes[ 1 ].set_xlabel( "CH1", fontsize=16 )

In [ ]:
run_interpolation = True
wave_threshold = 0.50 # relative
# wave_threshold_0 = 200 # ADC
# wave_threshold_1 = 600 # ADC
# wave_threshold_0 = 0.08 # V
# wave_threshold_1 = 0.3 # V

max_vals_ch0 = []
max_vals_ch1 = []
risetimes_ch0 = []
risetimes_ch1 = []

X_inter = None
if run_interpolation:
    X_inter = np.linspace( X[0], X[1023], 50*len(X) )

#descarta eventos que tenham zero ou mais de um pico acima do threshold
for evt_ in df_all.index:
    event_ch0_ = df_all.loc[ evt_ , 'Waveform_ch0']
    event_ch1_ = df_all.loc[ evt_ , 'Waveform_ch1']

    event_ch0_corr_ = corr_wf( event_ch0_ )
    event_ch1_corr_ = corr_wf( event_ch1_ )
    
    binX_ch0_ = None
    binX_ch1_ = None
    max_ch0_ = None
    max_ch1_ = None
    if run_interpolation:
        f_ch0_ = interpolate.interp1d(X, event_ch0_corr_, 'cubic')
        y_inter_ch0_ = f_ch0_( X_inter )
        f_ch1_ = interpolate.interp1d(X, event_ch1_corr_, 'cubic')
        y_inter_ch1_ = f_ch1_( X_inter )
        max_ch0_ = np.max( -event_ch0_corr_ )
        max_ch1_ = np.max( -event_ch1_corr_ )
        # Relative threshold
        binX_ch0_ = ( -y_inter_ch0_ > ( wave_threshold * max_ch0_ ) ).argmax()
        binX_ch1_ = ( -y_inter_ch1_ > ( wave_threshold * max_ch1_ ) ).argmax()
        # Absolute threshold
        # binX_ch0_ = ( -y_inter_ch0_ > ( wave_threshold_0 ) ).argmax()
        # binX_ch1_ = ( -y_inter_ch1_ > ( wave_threshold_1 ) ).argmax()
    else:
        max_ch0_ = np.max( event_ch0_corr_ )
        max_ch1_ = np.max( event_ch1_corr_ )
        # Relative threshold
        binX_ch0_ = ( -event_ch0_corr_ > ( wave_threshold * max_ch0_ ) ).argmax()
        binX_ch1_ = ( -event_ch1_corr_ > ( wave_threshold * max_ch1_ ) ).argmax()
        # Absolute threshold
        # binX_ch0_ = ( -y_inter_ch0_ > ( wave_threshold_0 ) ).argmax()
        # binX_ch1_ = ( -y_inter_ch1_ > ( wave_threshold_1 ) ).argmax()
    
    if binX_ch0_ > 0 and binX_ch1_ > 0:
        max_vals_ch0.append( max_ch0_ )
        max_vals_ch1.append( max_ch1_ )
        if run_interpolation:
            risetimes_ch0.append( X_inter[ binX_ch0_ ] )
            risetimes_ch1.append( X_inter[ binX_ch1_ ] )
        else:
            risetimes_ch0.append( X[ binX_ch0_ ] )
            risetimes_ch1.append( X[ binX_ch1_ ] )

max_vals_ch0 = np.array( max_vals_ch0 )
max_vals_ch1 = np.array( max_vals_ch1 )
risetimes_ch0 = np.array( risetimes_ch0 )
risetimes_ch1 = np.array( risetimes_ch1 )

print ( max_vals_ch0 )
print ( max_vals_ch1 )
print ( risetimes_ch0 )
print ( risetimes_ch1 )
print ( len(risetimes_ch0) )
print ( len(risetimes_ch1) )

In [ ]:
fig = plt.figure( figsize=(8,6) )
diff_times = ( risetimes_ch0 - risetimes_ch1 )
print ( diff_times )
plt.hist( diff_times, bins=20 )

In [ ]:
print ( np.std( diff_times ) )